In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import time
#原始目录
path = "../../data/AD/"
#结果目录
path2 = "../../data/AD/da/"
li = os.listdir(path)

exp_all_label_minisday   静态操作里面的ad_id 曝光值，非当天，16号label就是17号的（预测值）
exp_all_label_nowday     静态操作里面的ad_id 曝光值，当天，16号label就是16号实际的
exp_his_label            B榜测试数据里面的ad_id 在曝光数据里统计出来的历史曝光数据（直接用于测试集基于规则的填充）
op_sta_all_df            静态数据和操作数据的集合（并对所有天数进行了填充）
train1_with0             训练数据包含label为0数据，0占70%多
train1_with0_delete      训练数据（删除部分不适用于训练的数据，16号训练数据，但17号对广告修改了的不算）包含label为0数据，0占70%多
train1_without0          训练数据包含label没有0数据
train1_without0_delete   训练数据包含label没有0数据（删除部分不适用于训练的数据，16号训练数据，但17号对广告修改了的不算）包含label为0数据，0占70%多

## 1.读取曝光数据并去重

In [2]:
names = ['request_id','request_time','location_id','user_id','ad_id','ad_size','ad_bid','ad_pctr','ad_quality_ecpm','ad_totalEcpm']
totalExposureLog_file = path +'totalExposureLog.out'
exp_df = pd.read_csv(totalExposureLog_file,delimiter = '\t',parse_dates = ['request_time'],header=None,names = names)

In [3]:
exp_df.shape

(102386695, 10)

In [4]:
exp_df.drop_duplicates(keep='first', inplace=True)
exp_df.shape

(101507776, 10)

In [6]:
exp_df['request_time']=exp_df['request_time'].apply(lambda x:time.strftime("%Y%m%d",time.localtime(int(x))))

In [8]:
#	id	ad_id	create_time	goods_type	ad_account_id	ad_size	ad_bid	goods_id	ad_industry_id
test_df=pd.read_csv(path+'ad_test_data.csv')
sampleadlist=test_df.ad_id.unique()

## 2.读取B榜测试数据

In [339]:
testB_df_path = path +'Btest_sample_new.dat'
test_name=['id','ad_id','create_time','ad_size','ad_trade_id','good_class','good_id','ad_acc_id','deliver_time','people','ad_bid']
testB_df = pd.read_csv(testB_df_path,delimiter = '\t',parse_dates = ['create_time'],header=None,names = test_name)

In [340]:
testB_df.head()

,id,ad_id,create_time,ad_size,ad_trade_id,good_class,good_id,ad_acc_id,deliver_time,people,ad_bid
0,1,152978,1480387159,30,118,3,-1,23511,"281474976710655,281474976710655,28147497671065...","age:182,202,739,340,536,287,690,988,187,367,47...",37
1,2,507977,1551793928,30,122,13,629,4211,"70368744161280,70368744161280,70368744161280,7...",all,54
2,3,639943,1529565757,34,84,13,2525,15582,"281474976694272,281474976694272,28147497669427...","age:433,202,739,340,988,479,942,731,844,837,33...",161
3,4,605859,1553161934,40,98,13,3808,10907,"1099243192320,1099243192320,1099243192320,1099...",all,28
4,5,742582,1551329520,40,122,13,7017,15855,"274936627200,274936627200,274936627200,2749366...","age:433,819,79,522,601,753,347,829,741",12


## 3.统计B榜测试数据ad_id在曝光文件中出现的记录，统计ad_id每天出现的label(用于直接用规则填充测试集)

In [343]:
sampleadlist=testB_df.ad_id.unique()
exp_his_df=exp_df[exp_df['ad_id'].isin(sampleadlist)]
exp_his_df.shape

(826091, 10)

In [344]:
temp=exp_his_df.groupby(['ad_id','request_time','ad_bid'])['request_id'].count().reset_index()

In [345]:
exp_his_label=pd.DataFrame(temp.groupby(['ad_id','request_time'])['request_id'].sum().reset_index(),columns=['ad_id','request_time','request_id'])
exp_his_label.rename(columns=dict(zip(['ad_id','request_time','request_id'], ['ad_id','request_time','label'])), inplace=True)
exp_his_label.head()

,ad_id,request_time,label
0,415,20190217,6
1,415,20190218,1
2,415,20190219,2
3,415,20190220,2
4,415,20190221,2


In [346]:
exp_his_label.to_csv(path2+'exp_his_label.csv')

## 4.读取静态数据  去重，根据官方删除行业id，商品id，广告size问题数据

In [18]:
static_feature_names = ['ad_id','create_time','ad_acc_id','good_id','good_class','ad_trade_id','ad_size']
sta_df = pd.read_csv(path+'ad_static_feature.out',delimiter = '\t',parse_dates = ['create_time'],header=None,names = static_feature_names,dtype={'ad_id':int,"ad_acc_id": int,\
                              "good_id": str, "good_class": str, "ad_trade_id": str,'ad_size':str}) 

In [19]:
sta_df.shape

(735911, 7)

In [20]:
sta_df.drop_duplicates(keep='first',inplace=True)
sta_df.shape

(735911, 7)

In [22]:
sta_df=sta_df[~sta_df['ad_trade_id'].str.contains(',')]
sta_df.shape

(727720, 7)

In [23]:
sta_df=sta_df[~sta_df['good_id'].str.contains(',')]
sta_df.shape

(727717, 7)

In [25]:
sta_df=sta_df[~(sta_df['ad_size'].isnull())]
sta_df.shape

(502153, 7)

## 5.读取操作数据  去重，根据官方删除操作类型异常数据，删去2月30号数据
###   广告修改的字段列转行，并分组填充；清理频繁修改记录，只保留一天的最后一条修改记录

In [64]:
operation_names = ['ad_id','create_update_time','op_type','op_set','op_value']
op_df = pd.read_csv(path+'ad_operation.dat',delimiter = '\t',header=None,names = operation_names,dtype={"ad_id": int,'create_update_time':int,"op_type": int,"op_set": int, "op_value": object})

In [65]:
op_df.shape

(760866, 5)

In [66]:
op_df.drop_duplicates(keep='first', inplace=True)
op_df.shape

(760703, 5)

In [67]:
op_df=op_df[(op_df.create_update_time != 0) | (op_df.op_type != 1)]
op_df.shape

(759181, 5)

In [68]:
op_df=op_df[(op_df.create_update_time != 20190230000000)]
op_df.shape

(757889, 5)

In [69]:
op_df.head()

,ad_id,create_update_time,op_type,op_set,op_value
0,593323,0,2,2,90
1,593323,0,2,3,all
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,20190217000000,1,1,0
4,593323,20190218233855,1,2,90


In [70]:
op_df.drop_duplicates(['ad_id','create_update_time','op_type','op_set'],keep='first',inplace=True)
op_df.shape

(757282, 5)

In [71]:
op_df['index11']=op_df.apply(lambda x: str(x['ad_id'])+'-'+str(x['create_update_time'])+'-'+str(x['op_type']),axis=1)

In [72]:
temp=op_df.pivot(index='index11', columns='op_set', values='op_value')

In [73]:
temp.rename(index=str, columns={1: "status", 2: "ad_bid",3:"people",4:"deliver_time"},inplace=True)

In [74]:
temp.head()

op_set,status,ad_bid,people,deliver_time
index11,,,,
100007-0-2,NaN,200,"age:217,601,79,202,837,942,638,394,347,731,739...","281474976694272,281474976694272,28147497669427..."
100007-20190310154223-1,0,NaN,NaN,NaN
100007-20190312201207-1,1,NaN,NaN,NaN
100007-20190312201210-1,NaN,131,NaN,NaN
100007-20190312221003-1,0,NaN,NaN,NaN


In [75]:
op_df.drop_duplicates(subset=['ad_id','create_update_time','op_type'],keep='last',inplace=True)
op_df.shape

(680724, 6)

In [76]:
op_df=pd.merge(op_df,temp,how='left',left_on='index11',right_on='index11')

In [77]:
op_df.head()

,ad_id,create_update_time,op_type,op_set,op_value,index11,status,ad_bid,people,deliver_time
0,593323,0,2,4,"281474976710655,281474976710655,28147497671065...",593323-0-2,NaN,90,all,"281474976710655,281474976710655,28147497671065..."
1,593323,20190217000000,1,1,0,593323-20190217000000-1,0,NaN,NaN,NaN
2,593323,20190218233855,1,2,90,593323-20190218233855-1,NaN,90,NaN,NaN
3,593323,20190218000000,1,1,1,593323-20190218000000-1,1,NaN,NaN,NaN
4,593323,20190301000000,1,1,1,593323-20190301000000-1,1,NaN,NaN,NaN


In [78]:
op_df.drop(['op_set','op_value','index11'],axis=1,inplace=True)
op_df.sort_values(['ad_id','create_update_time'],inplace=True)

In [81]:
op_df.shape

(680724, 7)

In [83]:
#分组填充
op_data_fillna = pd.DataFrame()
ad_id_unique = op_df.ad_id.unique()
for ind in ad_id_unique:
    t = op_df.ad_id == ind
    a = op_df[t].fillna(axis=0,method='ffill')
    op_data_fillna = op_data_fillna.append(a)


In [85]:
op_data_fillna.head()

,ad_id,create_update_time,op_type,status,ad_bid,people,deliver_time
651230,31,0,2,NaN,100,area:11442,"281474976694272,281474976694272,28147497669427..."
651231,31,20190313195207,1,0,100,area:11442,"281474976694272,281474976694272,28147497669427..."
227403,32,0,2,NaN,83,area:7572,"70368475742208,70368475742208,70368475742208,7..."
227404,32,20190221053131,1,NaN,91,area:7572,"70368475742208,70368475742208,70368475742208,7..."
227405,32,20190221223403,1,0,91,area:7572,"70368475742208,70368475742208,70368475742208,7..."


In [106]:
op_df=op_data_fillna
#op_df.status.value_counts()
#   ???????
#删除广告状态无效行
op_df=op_df[op_df.status!='0']


In [107]:
op_df.shape

(402003, 7)

In [108]:
#清理频繁修改记录，只保留一天的最后一条修改记录
op_df.drop_duplicates(subset=['ad_id','create_update_time','op_type'],keep='last',inplace=True)
op_df.shape

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(402003, 7)

## 6.整合操作数据和静态数据
###  将静态数据中的修改或创建时间空的数据默认设置为很久之前，例如：20150407094355；并将操作数据时间字段中为0数据填充为静态数据时间字段

In [113]:
op_sta_df=pd.merge(op_df, sta_df, how='left', left_on='ad_id', right_on='ad_id')

In [156]:
#1552432970
time.strftime("%Y%m%d%H%M%S",time.localtime(int(1428399835)))

20150407094355


'20150407'

In [153]:
op_sta_df['create_time']=op_sta_df['create_time'].fillna('1428399835')
op_sta_df.create_time.isna().sum()

0

In [162]:
op_sta_df['create_time']=op_sta_df['create_time'].apply(lambda x:time.strftime("%Y%m%d%H%M%S",time.localtime(int(x))))

In [164]:
op_sta_df.loc[op_sta_df.create_update_time==0,'create_update_time']=op_sta_df[op_sta_df.create_update_time==0]['create_time'] 

In [173]:
op_sta_df.head()

,ad_id,create_update_time,op_type,ad_bid,people,deliver_time,ad_acc_id,good_id,good_class,ad_trade_id,ad_size
0,31,20190312232250,2,100,area:11442,"281474976694272,281474976694272,28147497669427...",12577.0,18683,13,224,40
1,32,20181130074017,2,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40
2,32,20190221053131,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40
3,32,20190222053139,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40
4,32,20190223053142,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40


In [169]:
op_sta_df.dropna(subset=['ad_size'],inplace=True)

In [172]:
op_sta_df.drop(['status','create_time'],axis=1,inplace=True)

## 7.筛选曝光数据ad_id在静态和操作整合数据出现的数据，并统计这些ad_id当日曝光

In [190]:
op_sta_idlist=op_sta_df.ad_id.unique()
exp_all_df=exp_df[exp_df['ad_id'].isin(op_sta_idlist)]
exp_all_df.shape

(5836700, 10)

In [193]:
temp=exp_all_df.groupby(['ad_id','request_time','ad_bid'])['request_id'].count().reset_index()
exp_all_label=pd.DataFrame(temp.groupby(['ad_id','request_time'])['request_id'].sum().reset_index(),columns=['ad_id','request_time','request_id'])
exp_all_label.rename(columns=dict(zip(['ad_id','request_time','request_id'], ['ad_id','request_time','label'])), inplace=True)
exp_all_label.head()

,ad_id,request_time,label
0,31,20190313,14
1,32,20190219,1
2,32,20190220,11
3,32,20190221,6
4,32,20190222,5


In [276]:
exp_all_label.to_csv(path2+'exp_all_label_nowday.csv')

## 8.生成训练集用得到的label,例如20190312 对应的label应当为0313号实际曝光值

In [278]:
def dayminis1(now):
    datestart=datetime.datetime.strptime(str(now),'%Y%m%d')
    datestart-=datetime.timedelta(days=+1)
    return datestart.strftime('%Y%m%d')
exp_all_label['request_time']=exp_all_label['request_time'].apply(lambda x:dayminis1(x))

In [279]:
exp_all_label.head()

,ad_id,request_time,label
0,31,20190312,14
1,32,20190218,1
2,32,20190219,11
3,32,20190220,6
4,32,20190221,5


In [280]:
exp_all_label.to_csv(path2+'exp_all_label_minisday.csv')

## 9.对操作和静态整合数据进行填充，即填充广告每天得状态（size,people...etc）
#### 例如 已有操作时间20190215  size:89   people:all .....
####                20190227  size:89   people:area123,12312 .....
####               则0215号到0227广告的状态就是持续0215的状态
####               问题1：20190227已经修改了状态，广告状态发生变化,则0226训练数据是否要进行删除（之后考虑）？？？？？？？？？？
### create_assist_date工具函数

In [195]:
kk=list(set(exp_all_label.request_time))
kk.sort()
print(kk[0])
kk[-1]

20190216


'20190319'

In [237]:
#生成某一段时间间隔的日期列表函数
import datetime
def create_assist_date(datestart = None,dateend = None):
	# 创建日期辅助表

	if datestart is None:
		datestart = '2016-01-01'
	if dateend is None:
		dateend = datetime.datetime.now().strftime('%Y%m%d')

	# 转为日期格式
	datestart=datetime.datetime.strptime(str(datestart),'%Y%m%d')
	dateend=datetime.datetime.strptime(str(dateend),'%Y%m%d')
	date_list = []
	date_list.append(datestart.strftime('%Y%m%d'))
	while datestart<dateend:
		# 日期叠加一天
	    datestart+=datetime.timedelta(days=+1)
	    # 日期转字符串存入列表
	    date_list.append(datestart.strftime('%Y%m%d'))
	return date_list
datelist=create_assist_date('20190215','20190318')
datelist

['20190215',
 '20190216',
 '20190217',
 '20190218',
 '20190219',
 '20190220',
 '20190221',
 '20190222',
 '20190223',
 '20190224',
 '20190225',
 '20190226',
 '20190227',
 '20190228',
 '20190301',
 '20190302',
 '20190303',
 '20190304',
 '20190305',
 '20190306',
 '20190307',
 '20190308',
 '20190309',
 '20190310',
 '20190311',
 '20190312',
 '20190313',
 '20190314',
 '20190315',
 '20190316',
 '20190317',
 '20190318']

In [201]:
#每个ad_id对应20190215~20190318，生成一个用于参考的举证
#笛卡尔积函数
def dikaerji(A,B):
    lenB = len(B)
#     print(lenB)
    dika_num = pd.DataFrame(columns=['ad_id','request_time'])
    for a in A:
        curA = np.array([a]*lenB)
        curA.shape = (lenB,1)   

        # 必须要先转换成np的aray形式，不然会报“没有shape”的错
        curB = np.array(B)
        curB.shape = (lenB,1)

        join_h = np.hstack((curA,curB))

        dika_num = dika_num.append(pd.DataFrame(join_h,columns=['ad_id','request_time']),ignore_index=True)
    return dika_num

dika_num=dikaerji(list(op_sta_idlist),datelist)
dika_num.head()

,ad_id,request_time
0,31,20190215
1,31,20190216
2,31,20190217
3,31,20190218
4,31,20190219


In [216]:
kk=list(map(int, list(set(op_sta_df.temp_time))))
kk.sort()
print(kk[0])
kk[-1]

20190215232250


20190318232250

In [255]:
op_sta_df['temp_time']=op_sta_df['create_update_time']
op_sta_df['temp_time']=op_sta_df['temp_time'].astype(str)
op_sta_df.loc[op_sta_df['temp_time']<'20190216','temp_time']='20190215232250'
op_sta_df.loc[op_sta_df['temp_time']>'20190318','temp_time']='20190318232250'
op_sta_df['temp_time']=op_sta_df['temp_time'].apply(lambda x:str(x)[0:8])

In [217]:
op_sta_df.columns

Index(['ad_id', 'create_update_time', 'op_type', 'ad_bid', 'people',
       'deliver_time', 'ad_acc_id', 'good_id', 'good_class', 'ad_trade_id',
       'ad_size', 'temp_time'],
      dtype='object')

In [242]:
#总共(静态数据)唯一要处理的ad_id数目
len(op_sta_idlist)

28916

In [272]:
#运行时间长，大概6个小时
op_sta_all_df=pd.DataFrame(columns=['ad_id','request_time','create_update_time','op_type','ad_bid','people','deliver_time', 'ad_acc_id', 'good_id', 'good_class', 'ad_trade_id','ad_size'])
for j,adid in enumerate(op_sta_idlist):
    if j % 2000==0:
        print(j)      
    temp=op_sta_df[op_sta_df.ad_id == adid].reset_index()
    le=len(temp)
    for i in range(le):
        if i == (le-1):
            temp2=pd.DataFrame(columns=['ad_id','request_time','create_update_time',
'ad_bid','people','deliver_time', 'ad_acc_id', 'good_id', 'good_class', 'ad_trade_id','ad_size'])
            temp2.create_update_time=create_assist_date(temp.loc[i,'temp_time'],'20190318')
            temp2.ad_id=adid
            temp2.op_type=temp.loc[i,'op_type']
            temp2.ad_bid=temp.loc[i,'ad_bid']
            temp2.people=temp.loc[i,'people']
            temp2.deliver_time=temp.loc[i,'deliver_time']
            temp2.ad_acc_id=temp.loc[i,'ad_acc_id']
            temp2.good_id=temp.loc[i,'good_id']
            temp2.good_class=temp.loc[i,'good_class']
            temp2.ad_trade_id=temp.loc[i,'ad_trade_id']
            temp2.ad_size=temp.loc[i,'ad_size']
            op_sta_all_df=pd.concat([op_sta_all_df,temp2],axis=0)
        else:
            temp2=pd.DataFrame(columns=['ad_id','request_time','create_update_time','ad_bid','people','deliver_time', 'ad_acc_id', 'good_id', 'good_class', 'ad_trade_id','ad_size'])
            temp2.create_update_time=create_assist_date(temp.loc[i,'temp_time'],temp.loc[i+1,'temp_time'])
            temp2.ad_id=adid
            temp2.op_type=temp.loc[i,'op_type']
            temp2.ad_bid=temp.loc[i,'ad_bid']
            temp2.people=temp.loc[i,'people']
            temp2.deliver_time=temp.loc[i,'deliver_time']
            temp2.ad_acc_id=temp.loc[i,'ad_acc_id']
            temp2.good_id=temp.loc[i,'good_id']
            temp2.good_class=temp.loc[i,'good_class']
            temp2.ad_trade_id=temp.loc[i,'ad_trade_id']
            temp2.ad_size=temp.loc[i,'ad_size']
            op_sta_all_df=pd.concat([op_sta_all_df,temp2],axis=0)

0


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000


In [273]:
op_sta_all_df.head()

,ad_acc_id,ad_bid,ad_id,ad_size,ad_trade_id,create_update_time,deliver_time,good_class,good_id,op_type,people,request_time
0,12577.0,100,31,40,224,20190312,"281474976694272,281474976694272,28147497669427...",13,18683,NaN,area:11442,NaN
1,12577.0,100,31,40,224,20190313,"281474976694272,281474976694272,28147497669427...",13,18683,NaN,area:11442,NaN
2,12577.0,100,31,40,224,20190314,"281474976694272,281474976694272,28147497669427...",13,18683,NaN,area:11442,NaN
3,12577.0,100,31,40,224,20190315,"281474976694272,281474976694272,28147497669427...",13,18683,NaN,area:11442,NaN
4,12577.0,100,31,40,224,20190316,"281474976694272,281474976694272,28147497669427...",13,18683,NaN,area:11442,NaN


In [277]:
op_sta_all_df.to_csv(path2+'op_sta_all_df.csv')

## 10。对填充后的数据进行清洗，存在填充异常的情况，暂时不管（生成时间函数问题，例如分别生成0215~0219；0219~0301,0219状态填充了两次）

In [286]:
op_sta_all_df.drop(['op_type','request_time'],axis=1,inplace=True)
op_sta_all_df.drop_duplicates(['ad_id','create_update_time'],keep='first', inplace=True)
op_sta_all_df=op_sta_all_df.reset_index()

In [324]:
exp_all_label.head()

,ad_id,request_time,label
0,31,20190312,14
1,32,20190218,1
2,32,20190219,11
3,32,20190220,6
4,32,20190221,5


In [325]:
exp_all_label.request_time.value_counts()

20190301    6690
20190313    6626
20190314    6509
20190302    6434
20190307    6396
20190312    6392
20190308    6320
20190228    6315
20190315    6249
20190304    6157
20190316    6123
20190311    6070
20190309    6054
20190306    6048
20190227    5990
20190305    5975
20190303    5962
20190310    5938
20190317    5891
20190318    5570
20190226    5410
20190225    5261
20190222    5245
20190223    5117
20190220    5080
20190221    5078
20190218    4904
20190224    4880
20190219    4845
20190217    4670
20190216    4476
20190215    1039
Name: request_time, dtype: int64

In [327]:
#保证填充后的数据比参考数据少
op_sta_all_df.create_update_time.value_counts()

20190318    28916
20190317    28133
20190316    27948
20190315    27622
20190314    26990
20190313    26174
20190312    25388
20190311    24720
20190310    24254
20190309    24021
20190308    23622
20190307    22993
20190306    22253
20190305    21402
20190304    20734
20190303    20340
20190302    20107
20190301    19707
20190228    18885
20190227    18331
20190226    17558
20190225    16844
20190224    16411
20190223    16151
20190222    15717
20190221    15093
20190220    14552
20190219    13930
20190218    13485
20190217    13078
20190216    12756
20190215    12379
Name: create_update_time, dtype: int64

In [350]:
dika_num.request_time.value_counts()

20190308    28916
20190306    28916
20190314    28916
20190316    28916
20190222    28916
20190302    28916
20190305    28916
20190312    28916
20190228    28916
20190315    28916
20190309    28916
20190226    28916
20190310    28916
20190220    28916
20190223    28916
20190318    28916
20190215    28916
20190303    28916
20190218    28916
20190301    28916
20190227    28916
20190304    28916
20190225    28916
20190221    28916
20190216    28916
20190313    28916
20190219    28916
20190307    28916
20190311    28916
20190224    28916
20190317    28916
20190217    28916
Name: request_time, dtype: int64

In [328]:
exp_all_label.rename(columns=dict(zip(['ad_id','request_time','label'], ['ad_id','create_update_time','label'])), inplace=True)

## 11.整合静态操作数据和label数据生成训练集
###  问题2：那么存在许多数据在某天曝光为0（可以计算某些ad_id在2月份到3月份总计曝光为0数据将ad_id删除）

In [329]:
df_train1=pd.merge(op_sta_all_df,exp_all_label,on=['ad_id','create_update_time'],how='left')

In [ ]:
df_train1['label'].fillna(0, inplace=True)

In [351]:
df_train1.head()

,ad_acc_id,ad_bid,ad_id,ad_size,ad_trade_id,create_update_time,deliver_time,good_class,good_id,people,label
0,12577.0,100,31,40,224,20190312,"281474976694272,281474976694272,28147497669427...",13,18683,area:11442,14.0
1,12577.0,100,31,40,224,20190313,"281474976694272,281474976694272,28147497669427...",13,18683,area:11442,0.0
2,12577.0,100,31,40,224,20190314,"281474976694272,281474976694272,28147497669427...",13,18683,area:11442,0.0
3,12577.0,100,31,40,224,20190315,"281474976694272,281474976694272,28147497669427...",13,18683,area:11442,0.0
4,12577.0,100,31,40,224,20190316,"281474976694272,281474976694272,28147497669427...",13,18683,area:11442,0.0


In [ ]:
temp=df_train1.groupby(['ad_id'])['label'].sum().reset_index()
valid_id=temp[temp.label!=0].ad_id.unique()
df_train1=df_train1[df_train1['ad_id'].isin(valid_id)]

In [373]:
df_train1.label.value_counts()

0.0       459062
1.0        43674
2.0        22046
3.0        14262
4.0        10065
5.0         7801
6.0         6285
7.0         5068
8.0         4278
9.0         3700
10.0        3279
11.0        2902
12.0        2591
13.0        2437
14.0        2098
15.0        1952
16.0        1711
17.0        1646
18.0        1583
19.0        1382
20.0        1291
21.0        1260
22.0        1125
23.0        1063
24.0         988
26.0         960
25.0         938
27.0         861
28.0         797
29.0         749
           ...  
1135.0         1
4537.0         1
2128.0         1
1129.0         1
4499.0         1
1121.0         1
4468.0         1
4467.0         1
4431.0         1
1107.0         1
4419.0         1
1102.0         1
4401.0         1
1099.0         1
1097.0         1
1095.0         1
628.0          1
1091.0         1
4361.0         1
740.0          1
1990.0         1
4347.0         1
4333.0         1
1083.0         1
4329.0         1
1082.0         1
4303.0         1
1073.0        

In [374]:
df_train1.to_csv(path2+'train1_with0.csv')

In [377]:
df_train1[df_train1['label']!=0].shape

(172108, 11)

In [378]:
df_train1[df_train1['label']!=0].to_csv(path2+'train1_without0.csv')

## 12.删除训练数据中问题数据
#### （例如16号训练数据（16号数据预测的应该是17号的，label也是17号的），但是17号该广告已经修改过，不能用于训练数据）

In [379]:
op_sta_df.head()

,ad_id,create_update_time,op_type,ad_bid,people,deliver_time,ad_acc_id,good_id,good_class,ad_trade_id,ad_size,temp_time
0,31,20190312232250,2,100,area:11442,"281474976694272,281474976694272,28147497669427...",12577.0,18683,13,224,40,20190312
1,32,20181130074017,2,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40,20190215
2,32,20190221053131,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40,20190221
3,32,20190222053139,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40,20190222
4,32,20190223053142,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40,20190223


In [391]:
op_sta_df['temp2_time']=op_sta_df['temp_time'].apply(lambda x:dayminis1(x))

In [392]:
op_sta_df.head()

,ad_id,create_update_time,op_type,ad_bid,people,deliver_time,ad_acc_id,good_id,good_class,ad_trade_id,ad_size,temp_time,temp2_time
0,31,20190312232250,2,100,area:11442,"281474976694272,281474976694272,28147497669427...",12577.0,18683,13,224,40,20190312,20190311
1,32,20181130074017,2,83,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40,20190215,20190214
2,32,20190221053131,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40,20190221,20190220
3,32,20190222053139,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40,20190222,20190221
4,32,20190223053142,1,91,area:7572,"70368475742208,70368475742208,70368475742208,7...",18752.0,32534,13,136,40,20190223,20190222


In [393]:
op_sta_df['index11']=op_sta_df.apply(lambda x: str(x['ad_id'])+'-'+str(x['temp2_time']),axis=1)
df_train1['index11']=df_train1.apply(lambda x: str(x['ad_id'])+'-'+str(x['create_update_time']),axis=1)

In [395]:
df_train1=df_train1[~df_train1['index11'].isin(op_sta_df.index11.unique())]

In [402]:
df_train1.drop(['index11'],axis=1,inplace=True)
df_train1=df_train1[['ad_id','create_update_time','ad_size','ad_trade_id','good_class','good_id','ad_acc_id','deliver_time','people','ad_bid','label']]

In [406]:
df_train1.to_csv(path2+'train1_with0_delete.csv')
df_train1.shape

(449704, 11)

In [405]:
df_train1[df_train1['label']!=0].to_csv(path2+'train1_without0_delete.csv')
df_train1[df_train1['label']!=0].shape

(81276, 11)

In [407]:
df_train1.head()

,ad_id,create_update_time,ad_size,ad_trade_id,good_class,good_id,ad_acc_id,deliver_time,people,ad_bid,label
0,31,20190312,40,224,13,18683,12577.0,"281474976694272,281474976694272,28147497669427...",area:11442,100,14.0
1,31,20190313,40,224,13,18683,12577.0,"281474976694272,281474976694272,28147497669427...",area:11442,100,0.0
2,31,20190314,40,224,13,18683,12577.0,"281474976694272,281474976694272,28147497669427...",area:11442,100,0.0
3,31,20190315,40,224,13,18683,12577.0,"281474976694272,281474976694272,28147497669427...",area:11442,100,0.0
4,31,20190316,40,224,13,18683,12577.0,"281474976694272,281474976694272,28147497669427...",area:11442,100,0.0
